In [1]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime
url = 'https://github.com/LeDataSciFi/data/raw/main/Firm%20Year%20Datasets%20(Compustat)/firms2020.csv'
firms_df = pd.read_csv(url).drop_duplicates('tic')

bod_df = pd.read_csv('./Input_data/BOD_Comp_Company_Fin.csv')
ceo_df = pd.read_csv('./Input_data/CEO_Comp_Fin.csv')
sp500_tickers = pd.read_csv('./Input_data/sp500_2022.csv')['Symbol'].tolist()
test_df = pd.read_csv('./Input_data/directorstest.csv')
df = pd.read_csv('./Input_data/ExecuComp_data.csv')

/var/folders/29/vl12lkgs28xgjpc6t0_jg9040000gn/T/ipykernel_1890/3821301156.py:13: DtypeWarning: Columns (80,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Input_data/ExecuComp_data.csv')


In [5]:
# bod_df = bod_df.drop(['NAICS', 'PCOMMFEES', 'PEXECDIRPD', 'EPSEX', 'EPSEXCHG', 'EPSEX3LS', 'EPSEX5LS', 'OIBD3LS'], axis=1)
# bod_df = bod_df.drop(['OIBDCHG', 'OIBD5LS', 'SALE3LS', 'SALE5LS', 'NI3LS', 'NI5LS', 'PDIRPENSN'], axis=1)
# bod_df = bod_df.drop(['NUMMTGS', 'TRS1YR', 'TRS3YR', 'TRS5YR', 'AJEX', 'BS_YIELD', 'BS_VOLATILITY'], axis=1)

BOD_df = bod_df[['GVKEY', 'YEAR', 'CUSIP', 'TICKER', 'SIC', 'ANNDIRRET', 
       'DIRMTGFEE', 'DIROPT', 'DIROPTAD', 'DIRSTK', 'DIRSTKAD', 
       'PDIRPENSN', 'OIBD', 'PRETAX', 'NIBEX', 'MKTVAL']]

In [3]:
ceo_df = ceo_df.drop(['CFOANN', 'PENSION_CHG', 'TOTAL_SEC', 'CHG_CTRL_PYMT', 'TOTAL_SEC_PCT', 'REPRICE', 'PCFO', 'EXECRANK'], axis=1)
ceo_df = ceo_df[ceo_df['CEOANN'].notnull()]
BOTH_df = ceo_df.merge(BOD_df, on = 'TICKER', how = 'left')

In [15]:
# BOTH_df.info()
# TDC1 is total compensation for all CEOs
#ANNDIRRET is for directors (their main salary) 


In [18]:
# assuming your DataFrame is named 'df'
BOTH_df['director_comp'] = (BOTH_df['ANNDIRRET'] + BOTH_df['DIRMTGFEE'] + BOTH_df['DIROPT'] 
                                   + BOTH_df['DIROPTAD'] + BOTH_df['DIRSTK'] + BOTH_df['DIRSTKAD'] + 
                                   (BOTH_df['PDIRPENSN'] * BOTH_df['ANNDIRRET'])
                                  )

In [23]:
grouped_df = BOTH_df.groupby('TICKER')['director_comp'].sum().reset_index()

merged_df = pd.merge(BOTH_df, grouped_df, on='TICKER', how='left')

merged_df = merged_df.rename(columns={'director_comp_y': 'total_director_comp'})


In [24]:
merged_df['total_director_comp'].describe()

count    59752.000000
mean      3312.646793
std       3424.132830
min          0.000000
25%        134.676000
50%       2363.900000
75%       5160.984000
max      20388.662000
Name: total_director_comp, dtype: float64

In [25]:
ceo_df = ceo_df[ceo_df['CEOANN'].notnull()]
